<a href="https://colab.research.google.com/github/azadkhah/speech2text/blob/main/ASR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

Fri Jun 10 07:33:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
import os
os.chdir('/')

In [67]:
!ls

bin	 datalab  home	 lib64	opt	    root  srv		     tmp    var
boot	 dev	  lib	 media	proc	    run   sys		     tools
content  etc	  lib32  mnt	python-apt  sbin  tensorflow-1.15.2  usr


In [6]:
%env LC_ALL=C.UTF-8
%env LANG=C.UTF-8
%env TRANSFORMERS_CACHE=/content/cache
%env HF_DATASETS_CACHE=/content/cache
%env CUDA_LAUNCH_BLOCKING=1

env: LC_ALL=C.UTF-8
env: LANG=C.UTF-8
env: TRANSFORMERS_CACHE=/content/cache
env: HF_DATASETS_CACHE=/content/cache
env: CUDA_LAUNCH_BLOCKING=1


In [7]:
!ls

cv-corpus-9.0-2022-04-27  cv-corpus-9.0-2022-04-27-fa.tar.gz


In [8]:
%%capture
!pip install -U datasets
!pip install -U transformers
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install hazm
!pip install num2fawords

In [61]:
%%capture
!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install wandb

In [9]:
%pip install pydub
%pip install pythainlp==2.3.1
%pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import os
import re
from typing import List, Dict, Tuple

import pandas as pd
from scipy.io import wavfile
from pythainlp.tokenize import word_tokenize


# ipython
import IPython.display as ipd
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm

In [11]:
# !wget https://mozilla-common-voice-datasets.s3.dualstack.us-west-2.amazonaws.com/cv-corpus-9.0-2022-04-27/cv-corpus-9.0-2022-04-27-fa.tar.gz

In [43]:
!tar -xzf cv-corpus-9.0-2022-04-27-fa.tar.gz

In [44]:
!rm -rf cv-corpus-9.0-2022-04-27-fa.tar.gz

In [45]:
from datasets import load_dataset, load_metric

import pandas as pd
import numpy as np

import hazm
from num2fawords import words, ordinal_words
from tqdm import tqdm

from sklearn.model_selection import train_test_split

import os
import string
import six
import re
import glob

In [46]:
_normalizer = hazm.Normalizer()

chars_to_ignore = [
    ",", "?", ".", "!", "-", ";", ":", '""', "%", "'", '"', "�",
    "#", "!", "؟", "?", "«", "»", "،", "(", ")", "؛", "'ٔ", "٬",'ٔ', ",", "?", 
    ".", "!", "-", ";", ":",'"',"“", "%", "‘", "”", "�", "–", "…", "_", "”", '“', '„',
    'ā', 'š',
#     "ء",
]

# In case of farsi
chars_to_ignore = chars_to_ignore + list(string.ascii_lowercase + string.digits)

chars_to_mapping = {
    'ك': 'ک', 'دِ': 'د', 'بِ': 'ب', 'زِ': 'ز', 'ذِ': 'ذ', 'شِ': 'ش', 'سِ': 'س', 'ى': 'ی',
    'ي': 'ی', 'أ': 'ا', 'ؤ': 'و', "ے": "ی", "ۀ": "ه", "ﭘ": "پ", "ﮐ": "ک", "ﯽ": "ی",
    "ﺎ": "ا", "ﺑ": "ب", "ﺘ": "ت", "ﺧ": "خ", "ﺩ": "د", "ﺱ": "س", "ﻀ": "ض", "ﻌ": "ع",
    "ﻟ": "ل", "ﻡ": "م", "ﻢ": "م", "ﻪ": "ه", "ﻮ": "و", 'ﺍ': "ا", 'ة': "ه",
    'ﯾ': "ی", 'ﯿ': "ی", 'ﺒ': "ب", 'ﺖ': "ت", 'ﺪ': "د", 'ﺮ': "ر", 'ﺴ': "س", 'ﺷ': "ش",
    'ﺸ': "ش", 'ﻋ': "ع", 'ﻤ': "م", 'ﻥ': "ن", 'ﻧ': "ن", 'ﻭ': "و", 'ﺭ': "ر", "ﮔ": "گ",
        
    # "ها": "  ها", "ئ": "ی",
    "۱۴ام": "۱۴ ام",
        
    "a": " ای ", "b": " بی ", "c": " سی ", "d": " دی ", "e": " ایی ", "f": " اف ",
    "g": " جی ", "h": " اچ ", "i": " آی ", "j": " جی ", "k": " کی ", "l": " ال ",
    "m": " ام ", "n": " ان ", "o": " او ", "p": " پی ", "q": " کیو ", "r": " آر ",
    "s": " اس ", "t": " تی ", "u": " یو ", "v": " وی ", "w": " دبلیو ", "x": " اکس ",
    "y": " وای ", "z": " زد ",
    "\u200c": " ", "\u200d": " ", "\u200e": " ", "\u200f": " ", "\ufeff": " ",
}


def multiple_replace(text, chars_to_mapping):
    pattern = "|".join(map(re.escape, chars_to_mapping.keys()))
    return re.sub(pattern, lambda m: chars_to_mapping[m.group()], str(text))

def remove_special_characters(text, chars_to_ignore_regex):
    text = re.sub(chars_to_ignore_regex, '', text).lower() + " "
    return text

def normalizer(text, chars_to_ignore=chars_to_ignore, chars_to_mapping=chars_to_mapping):
    chars_to_ignore_regex = f"""[{"".join(chars_to_ignore)}]"""
    text = text.lower().strip()

    text = _normalizer.normalize(text)
    text = multiple_replace(text, chars_to_mapping)
    text = remove_special_characters(text, chars_to_ignore_regex)
    text = re.sub(" +", " ", text)
    _text = []
    for word in text.split():
        try:
            word = int(word)
            _text.append(words(word))
        except:
            _text.append(word)
            
    text = " ".join(_text) + " "
    text = text.strip()

    if not len(text) > 0:
        return None
    
    return text + " "

In [47]:
abs_path_to_data = os.path.join("/content/drive/MyDrive/speech2text/cv-corpus-9.0-2022-04-27/fa")
!ls {abs_path_to_data}

clips	 invalidated.tsv  reported.tsv	train.tsv
dev.tsv  other.tsv	  test.tsv	validated.tsv


In [48]:
abs_path_to_data = os.path.join("/content/drive/MyDrive/speech2text", "cv-corpus-9.0-2022-04-27", "fa")
!ls {abs_path_to_data}/*.tsv

/content/drive/MyDrive/speech2text/cv-corpus-9.0-2022-04-27/fa/dev.tsv
/content/drive/MyDrive/speech2text/cv-corpus-9.0-2022-04-27/fa/invalidated.tsv
/content/drive/MyDrive/speech2text/cv-corpus-9.0-2022-04-27/fa/other.tsv
/content/drive/MyDrive/speech2text/cv-corpus-9.0-2022-04-27/fa/reported.tsv
/content/drive/MyDrive/speech2text/cv-corpus-9.0-2022-04-27/fa/test.tsv
/content/drive/MyDrive/speech2text/cv-corpus-9.0-2022-04-27/fa/train.tsv
/content/drive/MyDrive/speech2text/cv-corpus-9.0-2022-04-27/fa/validated.tsv


In [49]:
train_df = pd.concat([
    pd.read_csv(f"{abs_path_to_data}/train.tsv", sep="\t"),
    pd.read_csv(f"{abs_path_to_data}/dev.tsv", sep="\t")
])
_train_df = train_df.copy()
total_records = len(train_df)
train_df["id"] = range(0, total_records)
print(f"Step 0: {len(train_df)}")

train_df["path"] = abs_path_to_data + "/clips/" + train_df["path"]
train_df["status"] = train_df["path"].apply(lambda path: True if os.path.exists(path) else None)
train_df = train_df.dropna(subset=["path"])
train_df = train_df.drop("status", 1)
print(f"Step 1: {len(train_df)}")

train_df["sentence"] = train_df["sentence"].apply(lambda t: normalizer(t))
train_df = train_df.dropna(subset=["sentence"])
print(f"Step 2: {len(train_df)}")

term_a = set(list(range(0, total_records)))
term_b = set(train_df["id"].values.tolist())
removed_items_train = [_train_df.iloc[index]["path"] for index in list(term_a - term_b)]
train_df = train_df.reset_index(drop=True)
train_df.head()

Step 0: 34002


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  del sys.path[0]


Step 1: 34002
Step 2: 34002


,client_id,path,sentence,up_votes,down_votes,age,gender,accents,locale,segment,id
0,dfdeffdc55c8a41487b142ba5afb354d8eb081822ed326...,/content/drive/MyDrive/speech2text/cv-corpus-9...,او نسبت به هم مدرسهای خود محبت کودکانه داشت,2,0,NaN,NaN,NaN,fa,NaN,0
1,dfdeffdc55c8a41487b142ba5afb354d8eb081822ed326...,/content/drive/MyDrive/speech2text/cv-corpus-9...,او هرگونه اطلاع دربارهی توطئه را انکار کرد,2,0,NaN,NaN,NaN,fa,NaN,1
2,dfdeffdc55c8a41487b142ba5afb354d8eb081822ed326...,/content/drive/MyDrive/speech2text/cv-corpus-9...,اکنون میتوانی همهی نقشههای خود را آشکار کنی,2,0,NaN,NaN,NaN,fa,NaN,2
3,dfdeffdc55c8a41487b142ba5afb354d8eb081822ed326...,/content/drive/MyDrive/speech2text/cv-corpus-9...,اکثر پرندگان تخم میگذارند,2,0,NaN,NaN,NaN,fa,NaN,3
4,dfdeffdc55c8a41487b142ba5afb354d8eb081822ed326...,/content/drive/MyDrive/speech2text/cv-corpus-9...,با این تهدیدها نمیتوانی مرا بترسانی,2,0,NaN,NaN,NaN,fa,NaN,4


In [50]:
print(f"Items to be removed {len(removed_items_train)}")

Items to be removed 0


In [51]:
test_df = pd.read_csv(f"{abs_path_to_data}/test.tsv", sep="\t")

_test_df = test_df.copy()
total_records = len(test_df)
test_df["id"] = range(0, total_records)
print(f"Step 0: {len(test_df)}")

test_df["path"] = abs_path_to_data + "/clips/" + test_df["path"]
test_df["status"] = test_df["path"].apply(lambda path: True if os.path.exists(path) else None)
test_df = test_df.dropna(subset=["path"])
test_df = test_df.drop("status", 1)
print(f"Step 1: {len(test_df)}")

test_df["sentence"] = test_df["sentence"].apply(lambda t: normalizer(t))
test_df = test_df.dropna(subset=["sentence"])
print(f"Step 2: {len(test_df)}")

term_a = set(list(range(0, total_records)))
term_b = set(test_df["id"].values.tolist())
removed_items_test = [_test_df.iloc[index]["path"] for index in list(term_a - term_b)]
test_df = test_df.reset_index(drop=True)
test_df.head()

Step 0: 9852


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  # This is added back by InteractiveShellApp.init_path()


Step 1: 9852
Step 2: 9852


,client_id,path,sentence,up_votes,down_votes,age,gender,accents,locale,segment,id
0,016099f3eca09d2769a4bcbef9d90cb9742b9b2bf2b6ad...,/content/drive/MyDrive/speech2text/cv-corpus-9...,از مهمونداری کنار بکشم,2,1,NaN,NaN,NaN,fa,NaN,0
1,052f601fb7b8b1d14e87198a2507646c515721acbf4f67...,/content/drive/MyDrive/speech2text/cv-corpus-9...,برو از مهرداد بپرس,2,0,NaN,NaN,NaN,fa,NaN,1
2,08c6e3ae4acdced8e01b6cdd184ab85238211761114f12...,/content/drive/MyDrive/speech2text/cv-corpus-9...,خب تو چیکار می کنی,2,0,NaN,NaN,NaN,fa,NaN,2
3,0a65cb672c01a5dfb07d20c0a0b7621458edfbf85b3ed8...,/content/drive/MyDrive/speech2text/cv-corpus-9...,کسانی که مورد آزار قرار گرفته اند ذهنشان پر از...,2,0,thirties,male,NaN,fa,NaN,3
4,0b650fc0a5193006b0f32919910b372e7a85acd1cda35e...,/content/drive/MyDrive/speech2text/cv-corpus-9...,او تصمیم گرفت نقاشی را بیازماید,2,0,NaN,NaN,NaN,fa,NaN,4


In [52]:
print(f"Items to be removed {len(removed_items_test)}")

Items to be removed 0


In [53]:
text = " ".join(train_df["sentence"].values.tolist() + test_df["sentence"].values.tolist())
vocab = list(sorted(set(text)))

print(len(vocab), vocab)

38 [' ', '=', 'ء', 'آ', 'ئ', 'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ل', 'م', 'ن', 'ه', 'و', 'پ', 'چ', 'ژ', 'ک', 'گ', 'ی', 'ە']


In [58]:
import torchaudio
import librosa
import IPython.display as ipd
import numpy as np

sample = train_df.iloc[np.random.randint(0, len(train_df))]

path = sample["path"]
print(sample["sentence"], "\n")
speech = torchaudio.load(path)
speech = speech[0].numpy().squeeze()

speech = librosa.resample(np.asarray(speech), 48_000, 16_000)
ipd.Audio(data=np.asarray(speech), autoplay=True, rate=16000)

حراج بعد از نوروز  



In [78]:
train_df.to_csv("/content/train.csv", sep="\t", encoding="utf-8", index=False)
test_df.to_csv("/content/test.csv", sep="\t", encoding="utf-8", index=False)

print(train_df.shape)
print(test_df.shape)

(24150, 2)
(9852, 2)


In [72]:

# common_voice_train = load_dataset("csv", data_files={"train": "/content/train.csv"}, delimiter="\t")["train"]
# common_voice_test = load_dataset("csv", data_files={"test": "/content/test.csv"}, delimiter="\t")["test"]

print(common_voice_train)
print(common_voice_test)

NameError: ignored

In [17]:
cv_root: str = "./cv-corpus-9.0-2022-04-27"

train: pd.DataFrame = pd.read_csv(f"{cv_root}/fa/train.tsv", delimiter="\t")
train["set"] = "train"
dev: pd.DataFrame = pd.read_csv(f"{cv_root}/fa/dev.tsv", delimiter="\t")
dev["set"] = "dev"
test: pd.DataFrame = pd.read_csv(f"{cv_root}/fa/test.tsv", delimiter="\t")
test["set"] = "test"
data: pd.DataFrame = pd.read_csv(f"{cv_root}/fa/validated.tsv", delimiter="\t")

command: str = "sox {mp3_path} -t wav -r {sr} -c 1 -b 16 - |"

#     break
    
df = pd.DataFrame(train)
df.head()


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,client_id,path,sentence,up_votes,down_votes,age,gender,accents,locale,segment,set
0,dfdeffdc55c8a41487b142ba5afb354d8eb081822ed326...,common_voice_fa_25244256.mp3,او نسبت به هم مدرسهای خود محبت کودکانه داشت.,2,0,NaN,NaN,NaN,fa,NaN,train
1,dfdeffdc55c8a41487b142ba5afb354d8eb081822ed326...,common_voice_fa_25244257.mp3,او هرگونه اطلاع دربارهی توطئه را انکار کرد.,2,0,NaN,NaN,NaN,fa,NaN,train
2,dfdeffdc55c8a41487b142ba5afb354d8eb081822ed326...,common_voice_fa_25244258.mp3,اکنون میتوانی همهی نقشههای خود را آشکار کنی.,2,0,NaN,NaN,NaN,fa,NaN,train
3,dfdeffdc55c8a41487b142ba5afb354d8eb081822ed326...,common_voice_fa_25244259.mp3,اکثر پرندگان تخم میگذارند.,2,0,NaN,NaN,NaN,fa,NaN,train
4,dfdeffdc55c8a41487b142ba5afb354d8eb081822ed326...,common_voice_fa_25244260.mp3,با این تهدیدها نمیتوانی مرا بترسانی!,2,0,NaN,NaN,NaN,fa,NaN,train


In [18]:
print(f"Step 0: {len(df)}")

df["status"] = df["path"].apply(lambda path: True if os.path.exists(path) else None)
df = df.dropna(subset=["path"])
df = df.drop("status", 1)
print(f"Step 1: {len(df)}")

# df["prev_sentence"] = df["sentence"]
df["sentence"] = df["sentence"].apply(lambda t: normalizer(t))
df = df.dropna(subset=["sentence"])
print(f"Step 2: {len(df)}")

# df = df[["prev_sentence", "sentence", "path"]]
# df = df[["sentence", "path"]]
df = df.sample(frac=1)
df = df.reset_index(drop=True)
df.head()


Step 0: 24150


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """


Step 1: 24150
Step 2: 24150


,client_id,path,sentence,up_votes,down_votes,age,gender,accents,locale,segment,set
0,33adcf893ddfa9c17a21f6ec071ee84a5da73210cc903c...,common_voice_fa_23113803.mp3,تو دانشگاه که بودیم چند تا چیز خیلی مد بود,2,0,twenties,male,NaN,fa,NaN,train
1,b66de0d6017a4a2a656a8fc6bbe710ad496d33ab529a46...,common_voice_fa_25152857.mp3,ارتش نیرومند آلمان لهستان را درهم کوبید,2,0,thirties,male,NaN,fa,NaN,train
2,eda3ed57198d951d0f99a7506b8da2cc592b3456a3eab1...,common_voice_fa_30195861.mp3,پیشوا و پیرو,2,0,NaN,NaN,NaN,fa,NaN,train
3,b66de0d6017a4a2a656a8fc6bbe710ad496d33ab529a46...,common_voice_fa_25274027.mp3,احتمال موفقیت او را چگونه ارزیابی می کنید,2,0,thirties,male,NaN,fa,NaN,train
4,b8e506d7d9b5f2a9cfb4e08a088819f7f54bfd8c3e0ab8...,common_voice_fa_18226212.mp3,ولی جانسون واقعا حساسه,2,0,thirties,male,NaN,fa,NaN,train


In [19]:
main_vocab = ["ح", "چ", "ج", "ث", "ت", "پ", "ب", "آ", "ا", "ش", "س", "ژ", "ز", "ر", "ذ", "د", "خ", "ق", "ف", "غ", "ع", "ظ", "ط", "ض", "ص", "ی", "ه", "و", "ن", "م", "ل", "گ", "ک"]
text = " ".join(df["sentence"].values.tolist())
vocab = list(sorted(set(text)))

for v in main_vocab:
    if v not in vocab:
        print("v", v)

print(len(main_vocab), len(vocab))
print(vocab)

33 37
[' ', '=', 'ء', 'آ', 'ئ', 'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ل', 'م', 'ن', 'ه', 'و', 'پ', 'چ', 'ژ', 'ک', 'گ', 'ی']


In [62]:
import torchaudio
import librosa
import IPython.display as ipd
import numpy as np

idx = np.random.randint(0, len(df))
print(idx)
sample = df.iloc[idx]

path = sample["path"]
print(sample["sentence"], "\n")
speech, sr = torchaudio.load("/content/drive/MyDrive/speech2text/cv-corpus-9.0-2022-04-27/fa/clips/"+path)
speech = speech[0].numpy().squeeze()

speech = librosa.resample(np.asarray(speech), sr, 16_000)
ipd.Audio(data=np.asarray(speech), autoplay=True, rate=16000)

21340
فورا دکتر بیاورید  



In [75]:
train_df=pd.DataFrame(train)
test_df=pd.DataFrame(test)
train_df = train_df[["path", "sentence"]]
train_df = train_df.reset_index(drop=True)

test_df = test_df[["path", "sentence"]]
test_df = test_df.reset_index(drop=True)

print(train_df.shape)
print(test_df.shape)

(24150, 2)
(9852, 2)


In [76]:
save_path = "/".join(abs_path_to_data.split('/'))
print(save_path)

train_df.to_csv(f"{save_path}/train.csv", sep="\t", encoding="utf-8", index=False)
test_df.to_csv(f"{save_path}/test.csv", sep="\t", encoding="utf-8", index=False)

print(train_df.shape)
print(test_df.shape)

/content/drive/MyDrive/speech2text/cv-corpus-9.0-2022-04-27/fa
(24150, 2)
(9852, 2)


In [77]:
common_voice_train = load_dataset("csv", data_files={"train": "/content/drive/MyDrive/speech2text/cv-corpus-9.0-2022-04-27/fa/train.csv"}, delimiter="\t")["train"]
common_voice_test = load_dataset("csv", data_files={"test": "/content/drive/MyDrive/speech2text/cv-corpus-9.0-2022-04-27/fa/test.csv"}, delimiter="\t")["test"]

print(common_voice_train)
print(common_voice_test)

Using custom data configuration default-22f4a412f74be01c


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /content/cache/csv/default-22f4a412f74be01c/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-96244a26fd173e16


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /content/cache/csv/default-96244a26fd173e16/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['path', 'sentence'],
    num_rows: 24150
})
Dataset({
    features: ['path', 'sentence'],
    num_rows: 9852
})


In [79]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [82]:
show_random_elements(common_voice_train.remove_columns(["path"]), num_examples=20)

,sentence
0,بریده بریده خوابیدن
1,او جواهرات را در صندوق آهنی نگهداری میکند.
2,خط ناصاف
3,خندهی شاگردان شادی آنها را نشان میداد.
4,سرویس مبلمان
5,تکان زلزله
6,اگر شیطانی کنی پیش پدرت چغلی خواهم کرد.
7,او درسهای خود را در مکتب مصایب سخت زندگی یاد گرفت.
8,او از طنابی به دور گردنش آویزان بود.
9,این سگ هر کس را که به او نزدیک شود گاز میگیرد.


In [83]:
def normalizer(batch, chars_to_ignore=chars_to_ignore, chars_to_mapping=chars_to_mapping):
    chars_to_ignore_regex = f"""[{"".join(chars_to_ignore)}]"""
    text = batch["sentence"].lower().strip()

    text = _normalizer.normalize(text)
    text = multiple_replace(text, chars_to_mapping)
    text = remove_special_characters(text, chars_to_ignore_regex)
    text = re.sub(" +", " ", text)
    _text = []
    for word in text.split():
        try:
            word = int(word)
            _text.append(words(word))
        except:
            _text.append(word)
            
    text = " ".join(_text) + " "
    text = text.strip()

    if not len(text) > 0:
        return None
    
    batch["sentence"] = text
    
    return batch

In [84]:
print(common_voice_train[0]["sentence"])
print(common_voice_test[0]["sentence"])

او نسبت به هم مدرسهای خود محبت کودکانه داشت.
از مهمونداری کنار بکشم


In [86]:

common_voice_train = common_voice_train.map(normalizer, fn_kwargs={"chars_to_ignore": chars_to_ignore, "chars_to_mapping": chars_to_mapping})
common_voice_test = common_voice_test.map(normalizer, fn_kwargs={"chars_to_ignore": chars_to_ignore, "chars_to_mapping": chars_to_mapping})

  0%|          | 0/24150 [00:00<?, ?ex/s]

  0%|          | 0/9852 [00:00<?, ?ex/s]

In [87]:
print(common_voice_train[0]["sentence"])
print(common_voice_test[0]["sentence"])

او نسبت به هم مدرسهای خود محبت کودکانه داشت
از مهمونداری کنار بکشم


In [89]:
def extract_all_chars(batch):
    all_text = " ".join(batch["sentence"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

In [90]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [91]:
vocab_list = list(sorted(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0])))
vocab_list = [vocab for vocab in vocab_list if vocab not in [" ", "\u0307"]]
print(len(vocab_list))
print(vocab_list)

36
['=', 'ء', 'آ', 'ئ', 'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ل', 'م', 'ن', 'ه', 'و', 'پ', 'چ', 'ژ', 'ک', 'گ', 'ی']


In [92]:
vocab_list = list(sorted(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0])))
vocab_list = [vocab for vocab in vocab_list if vocab not in [" ", "\u0307"]]
print(len(vocab_list))
print(vocab_list)

36
['=', 'ء', 'آ', 'ئ', 'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ل', 'م', 'ن', 'ه', 'و', 'پ', 'چ', 'ژ', 'ک', 'گ', 'ی']


In [93]:
special_vocab = ["<pad>", "<s>", "</s>", "<unk>", "|"]

In [ ]:
vocab_dict = {v: k for k, v in enumerate(special_vocab + vocab_list)}
print(len(vocab_dict))
print(vocab_dict)

In [25]:
train_df = pd.concat([
    pd.read_csv(f"{abs_path_to_data}/train.tsv", sep="\t"),
    pd.read_csv(f"{abs_path_to_data}/dev.tsv", sep="\t")
])
_train_df = train_df.copy()
total_records = len(train_df)
train_df["id"] = range(0, total_records)
print(f"Step 0: {len(train_df)}")

train_df["path"] = abs_path_to_data + "/clips/" + train_df["path"]
train_df["status"] = train_df["path"].apply(lambda path: True if os.path.exists(path) else None)
train_df = train_df.dropna(subset=["path"])
train_df = train_df.drop("status", 1)
print(f"Step 1: {len(train_df)}")

train_df["sentence"] = train_df["sentence"].apply(lambda t: normalizer(t))
train_df = train_df.dropna(subset=["sentence"])
print(f"Step 2: {len(train_df)}")

term_a = set(list(range(0, total_records)))
term_b = set(train_df["id"].values.tolist())
removed_items_train = [_train_df.iloc[index]["path"] for index in list(term_a - term_b)]
train_df = train_df.reset_index(drop=True)
train_df.head()

Step 0: 34002


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  del sys.path[0]


Step 1: 34002
Step 2: 34002


,client_id,path,sentence,up_votes,down_votes,age,gender,accents,locale,segment,id
0,dfdeffdc55c8a41487b142ba5afb354d8eb081822ed326...,/content/drive/MyDrive/speech2text/cv-corpus-9...,او نسبت به هم مدرسهای خود محبت کودکانه داشت,2,0,NaN,NaN,NaN,fa,NaN,0
1,dfdeffdc55c8a41487b142ba5afb354d8eb081822ed326...,/content/drive/MyDrive/speech2text/cv-corpus-9...,او هرگونه اطلاع دربارهی توطئه را انکار کرد,2,0,NaN,NaN,NaN,fa,NaN,1
2,dfdeffdc55c8a41487b142ba5afb354d8eb081822ed326...,/content/drive/MyDrive/speech2text/cv-corpus-9...,اکنون میتوانی همهی نقشههای خود را آشکار کنی,2,0,NaN,NaN,NaN,fa,NaN,2
3,dfdeffdc55c8a41487b142ba5afb354d8eb081822ed326...,/content/drive/MyDrive/speech2text/cv-corpus-9...,اکثر پرندگان تخم میگذارند,2,0,NaN,NaN,NaN,fa,NaN,3
4,dfdeffdc55c8a41487b142ba5afb354d8eb081822ed326...,/content/drive/MyDrive/speech2text/cv-corpus-9...,با این تهدیدها نمیتوانی مرا بترسانی,2,0,NaN,NaN,NaN,fa,NaN,4


In [26]:
print(f"Items to be removed {len(removed_items_train)}")

Items to be removed 0


In [27]:
test_df = pd.read_csv(f"{abs_path_to_data}/test.tsv", sep="\t")

_test_df = test_df.copy()
total_records = len(test_df)
test_df["id"] = range(0, total_records)
print(f"Step 0: {len(test_df)}")

test_df["path"] = abs_path_to_data + "/clips/" + test_df["path"]
test_df["status"] = test_df["path"].apply(lambda path: True if os.path.exists(path) else None)
test_df = test_df.dropna(subset=["path"])
test_df = test_df.drop("status", 1)
print(f"Step 1: {len(test_df)}")

test_df["sentence"] = test_df["sentence"].apply(lambda t: normalizer(t))
test_df = test_df.dropna(subset=["sentence"])
print(f"Step 2: {len(test_df)}")

term_a = set(list(range(0, total_records)))
term_b = set(test_df["id"].values.tolist())
removed_items_test = [_test_df.iloc[index]["path"] for index in list(term_a - term_b)]
test_df = test_df.reset_index(drop=True)
test_df.head()

Step 0: 9852


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  # This is added back by InteractiveShellApp.init_path()


Step 1: 9852
Step 2: 9852


,client_id,path,sentence,up_votes,down_votes,age,gender,accents,locale,segment,id
0,016099f3eca09d2769a4bcbef9d90cb9742b9b2bf2b6ad...,/content/drive/MyDrive/speech2text/cv-corpus-9...,از مهمونداری کنار بکشم,2,1,NaN,NaN,NaN,fa,NaN,0
1,052f601fb7b8b1d14e87198a2507646c515721acbf4f67...,/content/drive/MyDrive/speech2text/cv-corpus-9...,برو از مهرداد بپرس,2,0,NaN,NaN,NaN,fa,NaN,1
2,08c6e3ae4acdced8e01b6cdd184ab85238211761114f12...,/content/drive/MyDrive/speech2text/cv-corpus-9...,خب تو چیکار می کنی,2,0,NaN,NaN,NaN,fa,NaN,2
3,0a65cb672c01a5dfb07d20c0a0b7621458edfbf85b3ed8...,/content/drive/MyDrive/speech2text/cv-corpus-9...,کسانی که مورد آزار قرار گرفته اند ذهنشان پر از...,2,0,thirties,male,NaN,fa,NaN,3
4,0b650fc0a5193006b0f32919910b372e7a85acd1cda35e...,/content/drive/MyDrive/speech2text/cv-corpus-9...,او تصمیم گرفت نقاشی را بیازماید,2,0,NaN,NaN,NaN,fa,NaN,4


In [28]:
print(f"Items to be removed {len(removed_items_test)}")


Items to be removed 0


In [29]:
text = " ".join(train_df["sentence"].values.tolist() + test_df["sentence"].values.tolist())
vocab = list(sorted(set(text)))

print(len(vocab), vocab)

38 [' ', '=', 'ء', 'آ', 'ئ', 'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ل', 'م', 'ن', 'ه', 'و', 'پ', 'چ', 'ژ', 'ک', 'گ', 'ی', 'ە']


In [73]:
import torchaudio
import librosa
import IPython.display as ipd
import numpy as np

sample = train_df.iloc[np.random.randint(0, len(train_df))]

path = sample["path"]
print(sample["sentence"], "\n")
speech = torchaudio.load(path)
speech = speech[0].numpy().squeeze()

speech = librosa.resample(np.asarray(speech), 48_000, 16_000)
ipd.Audio(data=np.asarray(speech), autoplay=True, rate=16000)

پشت خط زرد بایستید  

